-----

## 1\. Configuração do Ambiente e Chaves de API

Para que este notebook funcione, ele precisa se comunicar com a API do Google Gemini. Para isso, é necessário fornecer uma chave de API pessoal.

**Este notebook foi configurado para NÃO salvar sua chave diretamente no código, garantindo sua segurança.**

Siga os passos abaixo para configurar sua chave:

### Passo 1: Obtenha sua Chave da API do Gemini

1.  Acesse o **[Google AI Studio](https://makersuite.google.com/app/apikey)**.
2.  Faça login com sua conta Google.
3.  Clique em **"Create API key"** (Criar chave de API) para gerar uma nova chave.
4.  Copie a chave gerada. Ela é uma longa sequência de letras e números.

### Passo 2: Armazene a Chave no Colab Secrets

1.  Na barra lateral esquerda deste notebook, clique no ícone de chave ( **🔑** ) para abrir a aba de **"Secrets"** (Gerenciador de secrets).
2.  Clique em **"+ Adicionar novo secret"**.
3.  No campo `name` (nome), digite exatamente:
    ```
    GOOGLE_API_KEY
    ```
4.  No campo `value` (valor), cole a sua chave de API que você copiou do Google AI Studio.
5.  Ative o botão (slider) ao lado do nome para permitir que o notebook acesse este secret.

Pronto\! Agora, ao executar as células de código abaixo, ela irá carregar sua chave de forma segura sem nunca expô-la.

In [ ]:
%%capture
# 1. Instalações

# Esta célula instala todas as bibliotecas necessárias.

# NOTA: É comum o Colab gerar um AVISO sobre um "conflito de dependência"
# Este aviso é esperado e não impede a execução do projeto.

!pip install -q -U google-generativeai langchain-google-genai langchain requests

In [ ]:
# 2. Importações e Configuração da API

import os
import requests
import google.generativeai as genai
from google.colab import userdata

from google.genai import types

print("Configurando a chave da API do Google...")
try:
    # Carrega a chave de API a partir do 'Secrets' do Colab
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
    print("✅ Chave da API configurada com sucesso!")
except Exception as e:
    print(f"🚨 Erro ao configurar a chave: {e}")
    print("Verifique se você criou o secret 'GOOGLE_API_KEY' corretamente.")

Configurando a chave da API do Google...
✅ Chave da API configurada com sucesso!


In [ ]:
# 3. Teste de Conexão com o Gemini

print("Testando a conexão com a API do Gemini...")
try:
    for model in genai.list_models():
        if 'generateContent' in model.supported_generation_methods:
            print(model.name)
    print("\n✅ Conexão com o Gemini bem-sucedida!")
except Exception as e:
    print(f"🚨 Falha na conexão com o Gemini: {e}")

Testando a conexão com a API do Gemini...
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gem

In [ ]:
# 4. Teste de Conexão com a API da Steam

print("Testando a conexão com a API da Steam...")
# Usando o appid de 'Baldur's Gate 3' como exemplo
appid_teste = 1086940
url_steam = f"https://store.steampowered.com/appreviews/{appid_teste}"
params = {'json': 1, 'num_per_page': 1, 'language': 'brazilian'}

try:
    response = requests.get(url_steam, params=params)
    if response.status_code == 200:
        review_data = response.json()
        if review_data.get('success') == 1:
            print("✅ Conexão com a Steam bem-sucedida!")
            # Imprime um trecho da primeira review encontrada
            print("\nExemplo de review obtida:")
            print(review_data['reviews'][0]['review'][:300] + "...")
        else:
             print(f"🚨 A API da Steam respondeu com um erro: {review_data.get('query_summary')}")
    else:
        print(f"🚨 Erro na requisição HTTP: Status Code {response.status_code}")
except Exception as e:
    print(f"🚨 Falha na conexão com a Steam: {e}")

Testando a conexão com a API da Steam...
✅ Conexão com a Steam bem-sucedida!

Exemplo de review obtida:
Baldur's Gate 3: Uma Jornada Épica (100h / 50% Concluído)
Veredito Simples: Compre. Jogue. Prepare-se para perder a vida social.

Avaliação Sincera:

Depois de 100 horas e ainda com 50% do jogo pela frente (sim, a progressão é LENTA de um jeito maravilhoso), posso dizer: Baldur's Gate 3 (BG3) é...


# 2. Obtenção de Dados via API

In [ ]:
# 5. Função para obter reviews da Steam (Versão Simplificada)

def obter_reviews_steam(appid, quantidade=100):
    """
    Busca uma quantidade maior de reviews recentes de um jogo na Steam.
    Retorna uma lista contendo apenas os textos das reviews.
    """
    print(f"Buscando um lote de até {quantidade} reviews para o jogo ID: {appid}...")
    url = f"https://store.steampowered.com/appreviews/{appid}"
    params = {
        'json': 1,
        'filter': 'recent',
        'language': 'brazilian',
        'num_per_page': quantidade, # Pede um número maior de reviews
        'review_type': 'all'
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        lista_de_textos = [] # Retornará apenas os textos
        if data.get('success') == 1 and 'reviews' in data:
            for review_data in data['reviews']:
                # Adiciona apenas a string de texto da review
                lista_de_textos.append(review_data['review'])

            print(f"✅ {len(lista_de_textos)} reviews obtidas com sucesso!")
            return lista_de_textos
        else:
            print("🚨 A API da Steam não retornou reviews para este jogo.")
            return []
    except requests.exceptions.RequestException as e:
        print(f"🚨 Erro ao acessar a API da Steam: {e}")
        return []

# --- Exemplo de uso da função ---
# ID de Cyberpunk 2077: 1091500
# ID de Stardew Valley: 413150
id_do_jogo_alvo = 413150
lista_de_reviews = obter_reviews_steam(id_do_jogo_alvo, quantidade=5)

# Imprime um trecho da primeira review para verificar
if lista_de_reviews:
    print("\n--- Exemplo de Review ---")
    print(lista_de_reviews[0][:500] + "...")

Buscando um lote de até 5 reviews para o jogo ID: 413150...
✅ 5 reviews obtidas com sucesso!

--- Exemplo de Review ---
Perfeito...


# Aplicação das Técnicas de PLN

In [ ]:
# 6. Configuração do LangChain e Definição das Tarefas

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence

print("Inicializando o modelo Gemini...")

# --- Usando o nome de modelo mais estável ---
llm = ChatGoogleGenerativeAI(model="gemini-flash-latest",
                             temperature=0.2)

print("✅ Modelo LLM pronto para uso!")
print("\nCriando o template de prompt único...")

# --- PROMPT ÚNICO E OTIMIZADO ---
prompt_unico_template = """
Sua tarefa é fazer uma análise completa da review de um jogo da Steam. Siga as regras e o formato de saída estritamente.

REGRAS:
1.  **Seja Literal:** Extraia apenas pontos positivos/negativos que foram EXPLICITAMENTE mencionados. NÃO faça suposições.
2.  **Seja Conciso:** A "Breve Análise" deve ser um parágrafo curto resumindo o review. Os pontos devem ser tópicos simples (use "-").
3.  **Seja Limpo:** Se não houver pontos positivos ou negativos, escreva "Nenhum ponto mencionado."
4.  **Seja Preciso:** A "Classificação de Sentimento" deve ser apenas uma das três opções: Positivo, Negativo, ou Misto/Neutro.
5.  **Seja Estiloso:** Coloque os títulos "Breve Análise", "Pontos Positivos", "Pontos Negativos" e "Classificação de Sentimento" em negrito.

Review: "{texto_review}"

Breve Análise:
[Escreva um parágrafo curto aqui]

Pontos Positivos:
- [Liste os tópicos aqui]

Pontos Negativos:
- [Liste os tópicos aqui]

Classificação de Sentimento:
[Positivo, Negativo, ou Misto/Neutro]
"""

# Cria a chain única que será usada
prompt_unico = PromptTemplate.from_template(prompt_unico_template)
chain_unica = prompt_unico | llm


print("✅ Prompt e Chain únicos criados com sucesso!")

Inicializando o modelo Gemini...
✅ Modelo LLM pronto para uso!

Criando o template de prompt único...
✅ Prompt e Chain únicos criados com sucesso!


## 🎮 IDs de Jogos Famosos na Steam

Aqui está uma lista de referência rápida com os IDs de alguns jogos populares e aclamados na Steam, que podem ser usados para testar o analisador de reviews:

* **Counter-Strike 2:** `730`
* **Dota 2:** `570`
* **Baldur's Gate 3:** `1086940`
* **Elden Ring:** `1245620`
* **Grand Theft Auto V:** `271590`
* **Cyberpunk 2077:** `1091500`
* **Red Dead Redemption 2:** `1174180`
* **Stardew Valley:** `413150`
* **Terraria:** `105600`
* **PUBG: BATTLEGROUNDS:** `578080`
* **Megabonk:** `3405340`

---
**Como encontrar o ID de qualquer jogo:**
1.  Vá para a página do jogo na Loja Steam.
2.  Olhe a URL no seu navegador.
3.  O número no meio da URL é o ID.
*Exemplo: `https://store.steampowered.com/app/`**`413150`**`/Stardew_Valley/`*

In [ ]:
import time
import random
import textwrap
from IPython.display import Markdown
import re


# --- CONFIGURAÇÃO ---
id_do_jogo = 1091500
quantidade_para_buscar = 100
min_comprimento_review = 30    # Variável para controlar o filtro de tamanho

# --- Inicializa as variáveis que serão usadas por outras células ---
review_texto_global = ""
sentimento_global = ""

# --- OBTENÇÃO DOS DADOS ---
todas_as_reviews = obter_reviews_steam(id_do_jogo, quantidade=quantidade_para_buscar)

# --- Filtragem ---
# Filtra a lista completa, mantendo apenas reviews maiores que 'min_comprimento_review'
reviews_filtradas = [
    texto for texto in todas_as_reviews
    if len(texto.strip()) >= min_comprimento_review
]

# --- PROCESSAMENTO E ANÁLISE ALEATÓRIA ---

# --- LÓGICA PRINCIPAL ---
print(f"Buscando uma review aleatória de um total de {len(reviews_filtradas)} reviews...")

# --- ALEATORIEDADE ---
# Agora escolhe aleatoriamente da lista completa
review_texto = random.choice(reviews_filtradas)

# --- ANÁLISE COM O GEMINI ---
resultado_completo_obj = chain_unica.invoke({"texto_review": review_texto})
resultado_completo = resultado_completo_obj.content

# --- EXTRAÇÃO E ARMAZENAMENTO DAS INFORMAÇÕES ---

# 1. Salva o texto da review para a célula de áudio
review_texto_global = review_texto

# --- 2. EXTRAÇÃO DE SENTIMENTO (ROBUSTA - LINHA SEGUINTE) ---
sentimento_encontrado = None
linhas = resultado_completo.split('\n') # Divide em linhas uma vez

for i, linha in enumerate(linhas):
    # Encontra a linha do título
    if "classificação de sentimento" in linha.lower():
        # Tenta pegar a próxima linha
        if i + 1 < len(linhas):
            proxima_linha = linhas[i+1]
            # Limpa a *próxima* linha (remove *, espaços)
            sentimento_bruto = re.sub(r'[\*]', '', proxima_linha).strip()
            # Verifica se a linha limpa é um dos valores esperados
            if sentimento_bruto.lower() in ["positivo", "negativo", "misto/neutro"]:
                sentimento_encontrado = sentimento_bruto
                break # Encontrou, sai do loop
        # Se não conseguiu pegar a próxima linha ou ela não era válida, sai do loop
        break

# Define o sentimento_global com base no que foi encontrado
if sentimento_encontrado:
      sentimento_global = sentimento_encontrado
else:
    # Fallback se não encontrou
    sentimento_global = "Misto/Neutro"
    # Mantém o debug
    print(f"Aviso: Não foi possível encontrar o sentimento após a linha do título. Usando 'Neutro'.")

# --- CONSTRUÇÃO DO MARKDOWN ---
markdown_output = (f"""
---
## Análise de Review Aleatória do Jogo: `{id_do_jogo}`

---

### 📝 Texto Original
> {review_texto}

---

### 🤖 Resultados da Análise com Gemini

{resultado_completo}

---
""")


# Imprime uma mensagem de sucesso no console
print("✅ Análise concluída!")

Markdown(markdown_output)

Buscando um lote de até 100 reviews para o jogo ID: 1091500...
✅ 100 reviews obtidas com sucesso!
Buscando uma review aleatória de um total de 51 reviews...
✅ Análise concluída!



---
## Análise de Review Aleatória do Jogo: `1091500`

---

### 📝 Texto Original
> Bom, ainda tem alguns bugs, mas nada como no lançamento. Além disso, a história principal tem seus altos e baixos, eu até preferi as secundárias no geral. De resto, o jogo é bom.

---

### 🤖 Resultados da Análise com Gemini

**Breve Análise**:
O jogo é considerado bom no geral, tendo melhorado significativamente em relação aos bugs desde o lançamento. Embora a história principal seja inconsistente, o revisor destaca a qualidade das missões secundárias, que foram preferidas em comparação ao arco principal.

**Pontos Positivos**:
- O jogo é bom no geral.
- Os bugs diminuíram em comparação ao lançamento.
- As missões secundárias são boas/preferidas.

**Pontos Negativos**:
- Ainda possui alguns bugs.
- A história principal tem altos e baixos (inconsistente).

**Classificação de Sentimento**:
Positivo

---


# Áudio

In [ ]:
import wave
from IPython.display import Audio, display
import os
import google.generativeai as genai
from google.genai import types

# --- FUNÇÃO HELPER PARA SALVAR ÁUDIO ---
def save_audio_file(audio_data, filename="review_audio.wav"):
    """Salva os dados de áudio brutos em um arquivo .wav."""
    try:
        with wave.open(filename, "wb") as wf:
            wf.setnchannels(1)       # Mono
            wf.setsampwidth(2)       # 16-bit PCM
            wf.setframerate(24000)   # 24kHz, padrão do modelo TTS
            wf.writeframes(audio_data)
        return True
    except Exception as e:
        print(f"🚨 Erro ao salvar arquivo de áudio: {e}")
        return False

# --- GERAÇÃO DE ÁUDIO ---
if not review_texto_global or not sentimento_global:
    print("🚨 Nenhuma review analisada. Rode a célula de 'Análise de Texto' primeiro.")
else:
    try:
        # Define o tom da voz com base no sentimento salvo
        tom_da_voz = "calmo e neutro" # Padrão
        if sentimento_global == "Positivo":
            tom_da_voz = "animado e positivo"
        elif sentimento_global == "Negativo":
            tom_da_voz = "desapontado e crítico"

        print(f"Gerando áudio com sentimento '{sentimento_global}' (tom: {tom_da_voz})...")

        # Cria o prompt para o modelo TTS
        tts_prompt = f"Leia a seguinte review de forma {tom_da_voz}: {review_texto_global}"

        # Inicializa o modelo TTS (correto)
        tts_model = genai.GenerativeModel(model_name="gemini-2.5-flash-preview-tts")

        # --- CORREÇÃO: Passando a configuração como um DICIONÁRIO ---
        config_dict = {
            "response_modalities": ["AUDIO"],
            "speech_config": {
                "voice_config": {
                    "prebuilt_voice_config": {
                        "voice_name": "Orus"
                    }
                }
            }
        }

        # Chama o modelo TTS
        response = tts_model.generate_content(
            contents=tts_prompt,
            # Passa o dicionário em vez do objeto 'types'
            generation_config=config_dict
        )

        # Extrai o áudio
        audio_data = response.candidates[0].content.parts[0].inline_data.data

        # Salva o arquivo de áudio
        save_audio_file(audio_data, "review_audio.wav")
        print("✅ Áudio gerado e salvo como 'review_audio.wav'")

    except Exception as e:
        print(f"🚨 Falha ao gerar áudio: {e}")

# Exibe o player de áudio
if os.path.exists("review_audio.wav"):
    print("🔊 Áudio da Review (gerado com sentimento):")
    display(Audio("review_audio.wav"))
else:
    print("Nenhum arquivo de áudio ('review_audio.wav') encontrado. Rode a célula de geração de áudio primeiro.")

Gerando áudio com sentimento 'Positivo' (tom: animado e positivo)...
✅ Áudio gerado e salvo como 'review_audio.wav'
🔊 Áudio da Review (gerado com sentimento):


### 🎤 Lista de Vozes TTS Disponíveis (Modelos Gemini)

Você pode usar qualquer um dos nomes abaixo no parâmetro `voice_name`:

* `Achernar`
* `Achird`
* `Aoede`
* `Algenib`
* `Algieba`
* `Alnilam`
* `Autonoe`
* `Callirrhoe`
* `Charon`
* `Despina`
* `Enceladus`
* `Erinome`
* `Fenrir`
* `Gacrux`
* `Iapetus`
* `Kore`
* `Laomedeia`
* `Leda`
* `Orus`
* `Puck`
* `Pulcherrima`
* `Rasalgethi`
* `Sadachbia`
* `Sadaltager`
* `Schedar`
* `Sulafat`
* `Umbriel`
* `Vindemiatrix`
* `Zephyr`
* `Zubenelgenubi`